In [1]:
import numpy as np
import tables
from heartnet_v1 import reshape_folds
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
## import data
fold_dir = '/media/taufiq/Data1/heart_sound/feature/segmented_noFIR/folds_dec_2018/'
foldname = 'fold1_noFIR'

feat = tables.open_file(fold_dir + foldname + '.mat')
x_train = feat.root.trainX[:]
y_train = feat.root.trainY[0, :]
x_val = feat.root.valX[:]
y_val = feat.root.valY[0, :]
train_parts = feat.root.train_parts[:]
val_parts = feat.root.val_parts[0, :]

############## Relabeling ################

for i in range(0, y_train.shape[0]):
    if y_train[i] == -1:
        y_train[i] = 0  ## Label 0 for normal 1 for abnormal
for i in range(0, y_val.shape[0]):
    if y_val[i] == -1:
        y_val[i] = 0

############# Parse Database names ########

train_files = []
for each in feat.root.train_files[:][0]:
    train_files.append(chr(each))
print(len(train_files))
val_files = []
for each in feat.root.val_files[:][0]:
    val_files.append(chr(each))
print(len(val_files))

################### Reshaping ############

x_train, y_train, x_val, y_val = reshape_folds(x_train, x_val, y_train, y_val)
y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

76881
9639
(76881, 2500, 1)
(76881, 1)
(9639, 2500, 1)
(9639, 1)


In [98]:
meta_labels = np.asarray([ord(each)-97 for each in train_files])
for idx,each in enumerate(np.unique(train_files)):
    meta_labels[np.where(np.logical_and(y_train[:,0] == 1,np.asarray(train_files) == each))] = 6+idx
print(np.unique(meta_labels[y_train[:,0]==1]))

[ 6  7  8  9 10 11]


In [4]:
from AudioDataGenerator import BalancedAudioDataGenerator

In [99]:
datagen = BalancedAudioDataGenerator().flow(x_train,[y_train,train_files],
                                            meta_label=meta_labels,batch_size=120)
batchx, batchy = next(datagen)

Chunk size selected as 10


In [102]:
np.bincount(np.argmax(batchy[0],axis=-1))

array([60, 60])

In [114]:
np.bincount([ord(each)-97 for each in batchy[1]])

array([20, 20, 20, 20, 20, 20])

In [117]:
sum(np.asarray(train_files) == 'a')

11603